In [1]:
import os
import json
import base64
import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Local imports
from predict import predict_image
from prompt_templates import prompt_templates

# OpenRouter Claude API Setup (from previous notebook)
class OpenRouterClaudeAnalyzer:
    def __init__(self, api_key=None):
        key = api_key or os.getenv('OPENROUTER_API_KEY')
        if not key:
            raise ValueError("❌ OpenRouter API key required!")
        
        self.api_key = key
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
        self.model = "anthropic/claude-3-sonnet:beta"
        
        print("✅ OpenRouter Claude API initialized")
    
    def encode_image(self, image_path):
        try:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        except Exception as e:
            print(f"❌ Image encoding error: {e}")
            return None
    
    def get_image_media_type(self, image_path):
        ext = os.path.splitext(image_path)[1].lower()
        return {"jpg": "image/jpeg", "jpeg": "image/jpeg", "png": "image/png"}.get(ext[1:], "image/jpeg")
    
    def analyze_comparison(self, student_image_path, reference_image_path, comparison_prompt, max_retries=3):
        """Two images comparison with Claude"""
        student_b64 = self.encode_image(student_image_path)
        reference_b64 = self.encode_image(reference_image_path)
        
        if not student_b64 or not reference_b64:
            return {"error": "Failed to encode images"}
        
        student_media = self.get_image_media_type(student_image_path)
        reference_media = self.get_image_media_type(reference_image_path)
        
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "http://localhost:8888",
            "X-Title": "SAP Comparison Tool"
        }
        
        data = {
            "model": self.model,
            "max_tokens": 2000,
            "temperature": 0.1,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "**STUDENT SUBMISSION:**"},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:{student_media};base64,{student_b64}"}
                        },
                        {"type": "text", "text": "**REFERENCE SOLUTION (MUSTERLÖSUNG):**"},
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:{reference_media};base64,{reference_b64}"}
                        },
                        {"type": "text", "text": comparison_prompt}
                    ]
                }
            ]
        }
        
        for attempt in range(max_retries):
            try:
                response = requests.post(self.base_url, headers=headers, json=data, timeout=90)
                
                if response.status_code == 200:
                    result = response.json()
                    content = result.get('choices', [{}])[0].get('message', {}).get('content', '')
                    return {"response": content}
                else:
                    return {"error": f"HTTP {response.status_code}: {response.text}"}
                    
            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(5)
                    continue
                return {"error": f"Request failed: {str(e)}"}
        
        return {"error": "Max retries exceeded"}

# Initialize API
API_KEY = "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"

try:
    claude_analyzer = OpenRouterClaudeAnalyzer(api_key=API_KEY)
    print("🎯 Comparison engine ready!")
except Exception as e:
    print(f"❌ Setup failed: {e}")

print("✅ All imports and setup completed!")


✅ OpenRouter Claude API initialized
🎯 Comparison engine ready!
✅ All imports and setup completed!
